In [1]:
import os
import xarray as xr
import hvplot.xarray
import hvplot.pandas
import pandas as pd
import panel as pn
import intake
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geoviews as gv
import geopandas as gpd
import holoviews as hvs
from datetime import datetime
from holoviews.operation.datashader import rasterize
from holoviews import dim, opts

from monitor_texts import MonitoringAppTexts
from monitor_dates import MonitoringAppDates

gv.extension("bokeh")

monitor_app_texts = MonitoringAppTexts()

catalog_anl = intake.open_catalog('http://ftp1.cptec.inpe.br/pesquisa/das/carlos.bastarz/GSIMonitor/anls/catalog_anl.yml')
catalog_bkg = intake.open_catalog('http://ftp1.cptec.inpe.br/pesquisa/das/carlos.bastarz/GSIMonitor/anls/catalog_bkg.yml')

In [2]:
dsetanl = catalog_anl['2024033112']
dsetbkg = catalog_bkg['2024033112']

In [4]:
dset_anl = dsetanl.to_dask()
dset_bkg = dsetbkg.to_dask()

/home/carlos/miniconda3/envs/GSIMonitor/lib/python3.12/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


In [5]:
dset_anl

<xarray.Dataset> Size: 763MB
Dimensions:                                (lev: 64, lat: 450, lon: 900)
Coordinates:
  * lat                                    (lat) float32 2kB 89.69 ... -89.69
  * lev                                    (lev) float32 256B 1e+03 ... 0.1974
  * lon                                    (lon) float32 4kB 0.0 0.4 ... 359.6
Data variables: (12/28)
    CLOUD TOTAL PROGNOSTIC                 (lev, lat, lon) float32 104MB dask.array<chunksize=(8, 113, 225), meta=np.ndarray>
    DEEP SOIL TEMPERATURE                  (lat, lon) float32 2MB dask.array<chunksize=(225, 450), meta=np.ndarray>
    DIVERGENCE                             (lev, lat, lon) float64 207MB dask.array<chunksize=(8, 113, 225), meta=np.ndarray>
    ICE MIXING RATIO PROGNOSTIC            (lev, lat, lon) float32 104MB dask.array<chunksize=(8, 113, 225), meta=np.ndarray>
    LAND SEA ICE MASK                      (lat, lon) float32 2MB dask.array<chunksize=(225, 450), meta=np.ndarray>
    LIQ MIXING RATIO PROGNOSTIC            (lev, lat, lon) float32 104MB dask.array<chunksize=(8, 113, 225), meta=np.ndarray>
    ...                                     ...
    TEMPERATURE AT 2-M FROM SURFACE        (lat, lon) float32 2MB dask.array<chunksize=(225, 450), meta=np.ndarray>
    TOPOGRAPHY                             (lat, lon) float32 2MB dask.array<chunksize=(225, 450), meta=np.ndarray>
    VEGETATION COVER                       (lat, lon) float32 2MB dask.array<chunksize=(225, 450), meta=np.ndarray>
    VIRTUAL TEMPERATURE                    (lev, lat, lon) float64 207MB dask.array<chunksize=(8, 113, 225), meta=np.ndarray>
    VORTICITY                              (lat, lon) float32 2MB dask.array<chunksize=(225, 450), meta=np.ndarray>
    ZONAL WIND AT 10-M FROM SURFACE        (lat, lon) float32 2MB dask.array<chunksize=(225, 450), meta=np.ndarray>

In [6]:
dset_bkg

<xarray.Dataset> Size: 883MB
Dimensions:                                (lev: 64, lat: 450, lon: 900)
Coordinates:
  * lat                                    (lat) float32 2kB 89.69 ... -89.69
  * lev                                    (lev) float32 256B 1e+03 ... 0.1974
  * lon                                    (lon) float32 4kB 0.0 0.4 ... 359.6
Data variables: (12/39)
    CLOUD TOTAL PROGNOSTIC                 (lev, lat, lon) float32 104MB dask.array<chunksize=(8, 113, 225), meta=np.ndarray>
    CONVECTIVE PRECIPITATION               (lat, lon) float32 2MB dask.array<chunksize=(225, 450), meta=np.ndarray>
    DEEP SOIL TEMPERATURE                  (lat, lon) float32 2MB dask.array<chunksize=(225, 450), meta=np.ndarray>
    DIVERGENCE                             (lev, lat, lon) float64 207MB dask.array<chunksize=(8, 113, 225), meta=np.ndarray>
    DOWNWARD LONG WAVE AT BOTTOM           (lat, lon) float32 2MB dask.array<chunksize=(225, 450), meta=np.ndarray>
    DOWNWARD SHORT WAVE AT GROUND          (lat, lon) float32 2MB dask.array<chunksize=(225, 450), meta=np.ndarray>
    ...                                     ...
    UPWARD SHORT WAVE AT GROUND            (lat, lon) float32 2MB dask.array<chunksize=(225, 450), meta=np.ndarray>
    UPWARD SHORT WAVE AT TOP               (lat, lon) float32 2MB dask.array<chunksize=(225, 450), meta=np.ndarray>
    VEGETATION COVER                       (lat, lon) float32 2MB dask.array<chunksize=(225, 450), meta=np.ndarray>
    VIRTUAL TEMPERATURE                    (lev, lat, lon) float64 207MB dask.array<chunksize=(8, 113, 225), meta=np.ndarray>
    VORTICITY                              (lat, lon) float32 2MB dask.array<chunksize=(225, 450), meta=np.ndarray>
    ZONAL WIND AT 10-M FROM SURFACE        (lat, lon) float32 2MB dask.array<chunksize=(225, 450), meta=np.ndarray>

In [7]:
(dset_anl['TOPOGRAPHY'] - dset_bkg['TOPOGRAPHY']).compute().hvplot()

:Image   [lon,lat]   (TOPOGRAPHY)

In [ ]:
source = rasterize(gv.project(gv.Dataset(dset_anl['DIVERGENCE'].isel(lev=0)).to(gv.FilledContours, ['lon', 'lat'])))
source.opts(
          projection=ccrs.PlateCarree(),
          #cmap=cmap,
          #xlim=
          tools=["hover"],
          colorbar=True,
          symmetric=False,
          responsive=True,
          #title="Analysis of " + str(variable) + " @ level " + str(level) + " (valid for " + str(date) + ")",
          #width=width,
          height=400,
          #data_aspect=data_aspect,
          )
#source.redim(x=hvs.Dimension('x', range(-10, 10)))
#source.opts(clim=(-10, 10))
#ax_anl = source * gv.feature.coastline.options(line_width=2, line_color='k') * gv.feature.borders.options(line_width=2, line_color='k')

(ax_anl * gv.feature.coastline).opts(
    opts.FilledContours(levels=8, color_levels=10, cmap='nipy_spectral',
                        colorbar=True, width=600, height=300))#, projection=crs.Mollweide()))

In [ ]:
ax_anl

In [ ]:
source = rasterize(gv.project(gv.Dataset(dset_anl['DIVERGENCE'].isel(lev=0)).to(gv.Image, ['lon', 'lat'])))

In [ ]:
source.vdims

In [ ]:
float(dset_anl['DIVERGENCE'].isel(lev=0).min().values)